In [1]:
from pyspark.ml import feature
from pyspark.ml import clustering
from pyspark.ml import Pipeline
from pyspark.sql import functions as fn
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.sql import functions as sf
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.feature import RegexTokenizer
import requests
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import concat, col, lit, lower
from pyspark.sql.functions import isnan, when, count, col, isnull
from pyspark.sql.functions import concat_ws
from  pyspark.sql.functions import abs
# seting master("local[*]") enables multicore processing on all available logical cores on your machine
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [2]:
# Do not delete or change this cell

import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

In [3]:
airlines_df = spark.read.csv(get_training_filename('airlines.csv'), header=True, inferSchema=True)
airports_df = spark.read.csv(get_training_filename('airports.csv'), header=True, inferSchema=True)
flights_df = spark.read.csv(get_training_filename('flights.csv'), header=True, inferSchema=True)

In [4]:
shape = ((flights_df.count(), len(flights_df.columns)))
print('The shape of flights_df:', shape)

The shape of flights_df: (5819079, 31)

In [5]:
flights_df.select([count(when(isnull(c), c)).alias(c) for c in flights_df.columns]).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
 0| 0| 0| 0| 0| 0| 14721| 0| 0| 0| 86153| 86153| 89047| 89047| 6| 105071| 105071| 0| 92513| 92513| 0| 92513| 105071| 0| 0| 5729195| 4755640| 4755640| 4755640| 4755640| 4755640|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+

In [6]:
flights_df = flights_df.select('MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'CANCELLED')

flights_df = flights_df.filter((fn.col('CANCELLED')==0))

flights_df = flights_df.withColumn("Flight_Delayed", fn.when(fn.col("DEPARTURE_DELAY")<15, 0).otherwise(1))

from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=[ 0, 100, 1000, float('Inf') ],inputCol="DISTANCE", outputCol="Distance_Bucket")
flights_df = bucketizer.setHandleInvalid("keep").transform(flights_df)

from pyspark.sql.functions import udf
from pyspark.sql.types import *

t = {0.0:"Short", 1.0: "Medium", 2.0:"Long"}
udf_foo = udf(lambda x: t[x], StringType())
flights_df = flights_df.withColumn("Flight_Distance", udf_foo("Distance_Bucket"))

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="AIRLINE", outputCol="Airline_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCol="Airline_Numeric", outputCol="Airline_OHE")
flights_df= encoder.transform(flights_df)

indexer = StringIndexer(inputCol="ORIGIN_AIRPORT", outputCol="OA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="OA_Numeric", outputCol="Origin_Airport_OHE")
flights_df= encoder.transform(flights_df)

indexer = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="DA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="DA_Numeric", outputCol="Destination_Airport_OHE")
flights_df= encoder.transform(flights_df)

flights_df = flights_df.drop('ARRIVAL_DELAY')




In [7]:
from pyspark.sql import functions as fn
flights_df.groupBy('CANCELLED').agg(fn.count('*')).show()

+---------+--------+
CANCELLED|count(1)|
+---------+--------+
 1| 89884|
 0| 5729195|
+---------+--------+

In [8]:
flights_df.select("DISTANCE").rdd.max()[0]

Out[26]: 4983

In [9]:
flights_df.select("DISTANCE").rdd.min()[0]

Out[27]: 31

In [10]:
from pyspark.sql import functions as fn
flights_df.groupBy('Airline_Numeric').agg(fn.count('*')).show()

+---------------+--------+
Airline_Numeric|count(1)|
+---------------+--------+
 8.0| 194648|
 0.0| 1245812|
 7.0| 262772|
 1.0| 872057|
 4.0| 556746|
 11.0| 90248|
 3.0| 578393|
 2.0| 715065|
 10.0| 115375|
 13.0| 61369|
 6.0| 279607|
 5.0| 509150|
 9.0| 171852|
 12.0| 76101|
+---------------+--------+

In [11]:
flights_df.select('Airline_OHE').take(5)


Out[75]: [Row(Airline_OHE=SparseVector(13, {9: 1.0})),
 Row(Airline_OHE=SparseVector(13, {2: 1.0})),
 Row(Airline_OHE=SparseVector(13, {8: 1.0})),
 Row(Airline_OHE=SparseVector(13, {2: 1.0})),
 Row(Airline_OHE=SparseVector(13, {9: 1.0}))]

In [12]:
flights_df.limit(10).toPandas()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,CANCELLED,Flight_Delayed,Distance_Bucket,Flight_Distance,Airline_Numeric,Airline_OHE,OA_Numeric,Origin_Airport_OHE,DA_Numeric,Destination_Airport_OHE
0,1,1,4,AS,ANC,SEA,5,-11,1448,430,-22,0,0,2.0,Long,9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",65.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,1,4,AA,LAX,PBI,10,-8,2330,750,-9,0,0,2.0,Long,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",52.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,1,4,US,SFO,CLT,20,-2,2296,806,5,0,0,2.0,Long,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",6.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",14.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1,1,4,AA,LAX,MIA,20,-5,2342,805,-9,0,0,2.0,Long,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",24.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,1,4,AS,SEA,ANC,25,-1,1448,320,-21,0,0,2.0,Long,9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",66.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,1,1,4,DL,SFO,MSP,25,-5,1589,602,8,0,0,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,1,1,4,NK,LAS,MSP,25,-6,1299,526,-17,0,0,2.0,Long,10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,1,1,4,US,LAX,CLT,30,14,2125,803,-10,0,1,2.0,Long,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",14.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,1,1,4,AA,SFO,DFW,30,-11,1464,545,-13,0,0,2.0,Long,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,1,1,4,DL,LAS,ATL,30,3,1747,711,-15,0,0,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [13]:
flights_df.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- AIRLINE: string (nullable = true)
-- ORIGIN_AIRPORT: string (nullable = true)
-- DESTINATION_AIRPORT: string (nullable = true)
-- SCHEDULED_DEPARTURE: integer (nullable = true)
-- DEPARTURE_DELAY: integer (nullable = true)
-- DISTANCE: integer (nullable = true)
-- SCHEDULED_ARRIVAL: integer (nullable = true)
-- CANCELLED: integer (nullable = true)
-- Flight_Delayed: integer (nullable = false)
-- Distance_Bucket: double (nullable = true)
-- Flight_Distance: string (nullable = true)
-- Airline_Numeric: double (nullable = false)
-- Airline_OHE: vector (nullable = true)
-- OA_Numeric: double (nullable = false)
-- Origin_Airport_OHE: vector (nullable = true)
-- DA_Numeric: double (nullable = false)
-- Destination_Airport_OHE: vector (nullable = true)

In [14]:
training_df, testing_df = flights_df.randomSplit([0.9, 0.1], seed=5)

In [15]:
major_df = training_df.filter(col("Flight_Delayed") == 0)
minor_df = training_df.filter(col("Flight_Delayed") == 1)

In [16]:
ratio = major_df.count()/minor_df.count()
print(ratio)

4.4276492398259215

In [17]:
sampled_majority_df = major_df.sample(False, 1.2/ratio, seed=5)
combined_df_2 = sampled_majority_df.unionAll(minor_df)


In [18]:
testing_df.select([count(when(isnull(c), c)).alias(c) for c in training_df.columns]).show()

+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+-----------+----------+------------------+----------+-----------------------+
MONTH|DAY|DAY_OF_WEEK|AIRLINE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_DELAY|DISTANCE|SCHEDULED_ARRIVAL|CANCELLED|Flight_Delayed|Distance_Bucket|Flight_Distance|Airline_Numeric|Airline_OHE|OA_Numeric|Origin_Airport_OHE|DA_Numeric|Destination_Airport_OHE|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+-----------+----------+------------------+----------+-----------------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+-----------+----------+------------------+----------+-----------------------+

In [19]:
combined_df_2.groupBy('Flight_Delayed').agg(fn.count('*')).show()

+--------------+--------+
Flight_Delayed|count(1)|
+--------------+--------+
 1| 949914|
 0| 949450|
+--------------+--------+

In [20]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(
    inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED"], outputCol="features")

In [21]:
cols = ["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL"]

In [22]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import CrossValidatorModel
bce = BinaryClassificationEvaluator(labelCol='Flight_Delayed', metricName='areaUnderROC')
from pyspark.ml import evaluation
lr_evaluator_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="f1")
rf_evaluator_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="f1")

In [23]:
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.2, elasticNetParam=0.1, threshold=0.45, thresholds=[0.55, 0.45])
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)
#bce.evaluate(lr_pipeline.transform(testing_df))

In [24]:
bce.evaluate(lr_transform)

Out[25]: 0.6322203801446514

In [25]:
plt.figure(figsize=(10,6))
plt.plot([0, 1], [0, 1], 'r--')
plt.scatter(lr_model.stages[-1].summary.roc.select('FPR').collect(),
            lr_model.stages[-1].summary.roc.select('TPR').collect())
plt.title('ROC Scatter Plot : TPR/FPR')
plt.xlabel('FPR')
plt.ylabel('TPR')
display()

In [26]:
lr_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
lr_evaluator_recall.evaluate(lr_model.transform(testing_df))

Out[25]: 0.6036846099494937

In [27]:
predictions_train = lr_model.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.87 0.55 0.68 467809
 1 0.25 0.65 0.36 105586

 micro avg 0.57 0.57 0.57 573395
 macro avg 0.56 0.60 0.52 573395
weighted avg 0.76 0.57 0.62 573395

Out[19]: array([[258188, 209621],
 [ 37387, 68199]])

In [28]:
lr_evaluator_precision = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedPrecision")
lr_evaluator_precision.evaluate(lr_model.transform(testing_df))

Out[13]: 0.6000240385438204

In [29]:
lr_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="accuracy")
lr_evaluator_accuracy.evaluate(lr_model.transform(testing_df))

Out[29]: 0.5692184270877841

In [30]:
rf = RandomForestClassifier(featuresCol='features', labelCol='Flight_Delayed')
rf_pipeline = Pipeline(stages=[va, rf])
rf_model = rf_pipeline.fit(combined_df_2)
rf_transform = rf_model.transform(testing_df)
# rf_evaluator_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="f1")
# rf_evaluator_f1.evaluate(rf_transform)

In [31]:
rf_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
rf_evaluator_recall.evaluate(rf_transform)

Out[19]: 0.5661175978165139

In [32]:
rf_evaluator_Precision = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedPrecision")
rf_evaluator_Precision.evaluate(rf_transform)

Out[20]: 0.7631986378653461

In [33]:
rf_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="accuracy")
rf_evaluator_accuracy.evaluate(rf_transform)

Out[21]: 0.5661175978165139

In [34]:
rf_bce = BinaryClassificationEvaluator(labelCol='Flight_Delayed', metricName='areaUnderROC')
rf_bce.evaluate(rf_transform)

Out[22]: 0.6476120227457359

In [35]:
randomForest_model = rf_model.stages[-1]
RF_df = pd.DataFrame(list(zip(cols, randomForest_model.featureImportances.toArray())),
             columns = ['column', 'weight']).sort_values('weight', ascending=False)

In [36]:
ax1 = sns.barplot(x='weight', y='column', data=RF_df)
ax1.set(xlabel='Importance', ylabel='Features')
display()

In [37]:
gbt = GBTClassifier(featuresCol='features', labelCol='Flight_Delayed', maxIter=25, maxBins=15, stepSize=0.08, maxDepth=4)
gbt_pipeline = Pipeline(stages=[va, gbt])
gbt_model = gbt_pipeline.fit(combined_df_2)
gbt_transform = gbt_model.transform(testing_df)

In [38]:
predictions_train = gbt_transform
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.89 0.60 0.71 467809
 1 0.27 0.66 0.38 105586

 micro avg 0.61 0.61 0.61 573395
 macro avg 0.58 0.63 0.55 573395
weighted avg 0.77 0.61 0.65 573395

Out[14]: array([[278532, 189277],
 [ 35961, 69625]])

In [39]:
boost_model = gbt_model.stages[-1]
pd.DataFrame(list(zip(flights_df.columns[:20], boost_model.featureImportances.toArray())),
             columns = ['column', 'weight']).sort_values('weight', ascending=False)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3074076617825814> in <module> 
 ----> 1 boost_model = gbt_model . stages [ - 1 ] 
 2 pd.DataFrame(list(zip(flights_df.columns[:20], boost_model.featureImportances.toArray())),
 3 columns = ['column', 'weight']).sort_values('weight', ascending=False)

 NameError : name 'gbt_model' is not defined

In [40]:
gbt_bce = BinaryClassificationEvaluator(labelCol='Flight_Delayed', metricName='areaUnderROC')
gbt_bce.evaluate(gbt_transform)

Out[15]: 0.6790354239863563

In [41]:
gbt_evaluator_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="f1")
gbt_evaluator_f1.evaluate(gbt_transform)

Out[14]: 0.6602030484689703

In [42]:
gbt_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="accuracy")
gbt_evaluator_accuracy.evaluate(gbt_transform)

Out[16]: 0.6071852736769592

In [43]:
gbt_evaluator_precision = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedPrecision")
gbt_evaluator_precision.evaluate(gbt_transform)

Out[16]: 0.7740484152765228

In [44]:
gbt_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
gbt_evaluator_recall.evaluate(gbt_transform)

Out[17]: 0.6173510407310842

In [45]:
paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxBins, [25, 28, 31])\
  .addGrid(gbt.maxDepth, [4, 6, 8])\
  .addGrid(gbt.maxIter, [10, 20, 30])\
  .build()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3455463568524264> in <module> 
 ----> 1 paramGrid = ParamGridBuilder ( ) \ 
 2 . addGrid ( gbt . maxBins , [ 25 , 28 , 31 ] ) \ 
 3 . addGrid ( gbt . maxDepth , [ 4 , 6 , 8 ] ) \ 
 4 . addGrid ( gbt . maxIter , [ 10 , 20 , 30 ] ) \ 
 5 . build ( ) 

 NameError : name 'ParamGridBuilder' is not defined

In [46]:
cv = CrossValidator()\
  .setEstimator(gbt)\
  .setEvaluator(gbt_bce)\
  .setEstimatorParamMaps(paramGrid)\
  .setNumFolds(3)

In [47]:
sample_cv = combined_df_2.sample(True, 0.2,seed=5)

In [48]:
cv_pipeline = Pipeline(stages=[va,cv])
cv_model = cv_pipeline.fit(sample_cv)
cv_transform = cv_model.transform(testing_df)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [49]:
#rf_cv_evaluator_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="f1")
gbt_bce.evaluate(cv_transform)

In [50]:
from pyspark.ml import classification

mlp = classification.MultilayerPerceptronClassifier(featuresCol='features', labelCol='Flight_Delayed', layers=[4,2])
mlp_pipeline = Pipeline(stages=[va, mlp]).fit(training_df)
evaluator.evaluate(mlp_pipeline.transform(testing_df))

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4117519078817409> in <module> 
 3 mlp = classification . MultilayerPerceptronClassifier ( featuresCol = 'features' , labelCol = 'Flight_Delayed' , layers = [ 4 , 5 , 4 , 2 ] ) 
 4 mlp_pipeline = Pipeline ( stages = [ va , mlp ] ) . fit ( training_df ) 
 ----> 5 evaluator . evaluate ( mlp_pipeline . transform ( testing_df ) ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in evaluate (self, dataset, params) 
 69 return self . copy ( params ) . _evaluate ( dataset ) 
 70 else : 
 ---> 71 return self . _evaluate ( dataset ) 
 72 else : 
 73 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in _evaluate (self, dataset) 
 99 """
 100 self . _transfer_params_to_java ( ) 
 --> 101 return self . _java_obj . evaluate ( dataset . _jdf ) 
 102 
 103 def isLargerBetter ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1065.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 735.0 failed 1 times, most recent failure: Lost task 7.0 in stage 735.0 (TID 3690, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:191)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:62)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:99)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:140)
	at org.apache.spark.scheduler.Task.run(Task.scala:113)
	at org.apache.spark.executor.Executor$TaskRunner$$anonfun$13.apply(Executor.scala:537)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1541)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:543)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: A & B Dimension mismatch!
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.ml.ann.

In [51]:
from pyspark.ml.classification import LinearSVC

svm = LinearSVC(featuresCol='features', labelCol='Flight_Delayed', maxIter=10, regParam=0.1)
svm_pipeline = Pipeline(stages=[va, svm]).fit(training_df)
evaluator.evaluate(svm_pipeline.transform(testing_df))


Out[22]: 0.7754379526967401

In [52]:
from pyspark.ml import feature

pca = PCA(k=2, inputCol="features", outputCol="pca_features")
 model = pca.fit(df)
 model.transform(df).collect()

In [53]:
flights_df_sample = flights_df.sample(True, 0.5, 42)

In [54]:
shape = ((flights_df_sample.count(), len(flights_df_sample.columns)))
print('The shape of flights_df_sample:', shape)

The shape of flights_df_sample: (2862296, 20)

In [55]:
flights_df_sample.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- AIRLINE: string (nullable = true)
-- ORIGIN_AIRPORT: string (nullable = true)
-- DESTINATION_AIRPORT: string (nullable = true)
-- SCHEDULED_DEPARTURE: integer (nullable = true)
-- DEPARTURE_DELAY: integer (nullable = true)
-- DISTANCE: integer (nullable = true)
-- SCHEDULED_ARRIVAL: integer (nullable = true)
-- CANCELLED: integer (nullable = true)
-- Flight_Delayed: integer (nullable = false)
-- Distance_Bucket: double (nullable = true)
-- Flight_Distance: string (nullable = true)
-- Airline_Numeric: double (nullable = false)
-- Airline_OHE: vector (nullable = true)
-- OA_Numeric: double (nullable = false)
-- Origin_Airport_OHE: vector (nullable = true)
-- DA_Numeric: double (nullable = false)
-- Destination_Airport_OHE: vector (nullable = true)

In [56]:
flights_df_sample_pandas = flights_df_sample.toPandas()

/databricks/spark/python/pyspark/sql/dataframe.py:2163: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)

In [57]:
flights_df_sample.groupBy('Flight_Delayed').agg(fn.count('*')).show()

+--------------+--------+
Flight_Delayed|count(1)|
+--------------+--------+
 1| 646606|
 0| 2215690|
+--------------+--------+

In [58]:
flights_df.groupBy('Flight_Delayed').agg(fn.count('*')).show()

+--------------+--------+
Flight_Delayed|count(1)|
+--------------+--------+
 1| 1292586|
 0| 4436609|
+--------------+--------+

In [59]:
from pyspark.ml.stat import Correlation

# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED", "Flight_Delayed"], outputCol=vector_col)
df_vector = assembler.transform(flights_df).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(df_vector, vector_col)


In [60]:
matrix.show()

+----------------------+
pearson(corr_features)|
+----------------------+
 1.0 ...|
+----------------------+

In [61]:
sns.distplot(flights_df_sample_pandas['MONTH'])
display()

In [62]:
sns.distplot(flights_df_sample_pandas['DAY'])
display()

In [63]:
sns.distplot(flights_df_sample_pandas['DAY_OF_WEEK'])
display()

In [64]:
flights_df_sample_pandas['AIRLINE'].value_counts().plot(kind='bar')
display()

In [65]:
airline_count  = flights_df_sample_pandas['AIRLINE'].value_counts()
airline_count = airline_count[:10,]
plt.figure(figsize=(10,5))
sns.barplot(airline_count.index, airline_count.values, alpha=0.8)
display()

In [66]:
oa_count  = flights_df_sample_pandas['ORIGIN_AIRPORT'].value_counts()
oa_count = oa_count[:10,]
plt.figure(figsize=(10,5))
sns.barplot(oa_count.index, oa_count.values, alpha=0.8)
display()

In [67]:
da_count  = flights_df_sample_pandas['DESTINATION_AIRPORT'].value_counts()
da_count = da_count[:10,]
plt.figure(figsize=(10,5))
sns.barplot(da_count.index, da_count.values, alpha=0.8)
display()

In [68]:
sns.distplot(flights_df_sample_pandas['SCHEDULED_DEPARTURE'])
display()

In [69]:
sns.distplot(flights_df_sample_pandas['DEPARTURE_DELAY'])
display()

In [70]:
sns.distplot(flights_df_sample_pandas['DISTANCE'])
display()

In [71]:
sns.distplot(flights_df_sample_pandas['Distance_Bucket'])
display()

In [72]:
sns.distplot(flights_df_sample_pandas['SCHEDULED_ARRIVAL'])
display()

In [73]:
sns.distplot(flights_df_sample_pandas['CANCELLED'])
display()

In [74]:
sns.distplot(flights_df_sample_pandas['Flight_Delayed'])
display()

In [75]:

sns.barplot(x="MONTH", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [76]:
df3 = flights_df.groupBy('MONTH').avg('DEPARTURE_DELAY').toPandas()
df3
fig4 = sns.barplot(x = 'MONTH', y = 'avg(DEPARTURE_DELAY)', data = df3)
display()

In [77]:
df3.head()

,MONTH,avg(DEPARTURE_DELAY)
0,12,11.738865
1,1,9.732780
2,6,13.942455
3,3,9.627705
4,5,9.418682


In [78]:
sns.barplot(x="MONTH", y="DEPARTURE_DELAY", hue="Flight_Distance", data=flights_df_sample_pandas)
display()

In [79]:
sns.barplot(x="DAY", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [80]:
sns.barplot(x="DAY", y="DEPARTURE_DELAY", hue="Flight_Distance", data=flights_df_sample_pandas)
display()

In [81]:
sns.barplot(x="DAY_OF_WEEK", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [82]:
sns.barplot(x="DAY_OF_WEEK", y="DEPARTURE_DELAY", hue="Flight_Distance", data=flights_df_sample_pandas)
display()

In [83]:
sns.barplot(x="AIRLINE", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [84]:
sns.barplot(x="ORIGIN_AIRPORT", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [85]:
sns.barplot(x="SCHEDULED_DEPARTURE", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [86]:
sns.barplot(x="Distance_Bucket", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [87]:
sns.barplot(x="AIRLINE", y="DEPARTURE_DELAY", hue="Flight_Distance", data=flights_df_sample_pandas)
display()